# Pryngles module: Spangling

This module contains all the physics of light scattered on spangles

In [1]:
from pryngles import *

## External modules

In [2]:
import pandas as pd
from collections import OrderedDict as odict
from copy import deepcopy

## The Spangling class

This class contains a family of routines useful for spangling

In [3]:
Spangling_doc=\
"""A Spangling associated to an object or set of objects.
    
Initialization attributes:

    nspangles: int, default = 1:
        Number of spangles in spangling.
        
    df : Pandas DataFrame: 
        Dataframe containing all the information about the spangling.
        For Columns see global variable SPANGLING_COLUMNS.
        
Secondary attributes:

    default: dictionary, default = SPANGLING_COLUMNS: 
        Hash of the object, ie. a unique string identifying the object 
        (see hash Python function)

Public methods:

    update_body(props):
        Update a given property.
"""

In [4]:
#Columns of spangling
SPANGLING_COLUMNS=odict(
    {
        "body_hash":"",
        #Type of spangle
        "type":SOLID_SPANGLE,
        #Cartesian coordinates
        "x_equ":0,"y_equ":0,"z_equ":0,
        "x_ecl":0,"y_ecl":0,"z_ecl":0,
        "x_obs":0,"y_obs":0,"z_obs":0,
        #Spherical coordinates
        "r_equ":0,"t_equ":0,"f_equ":0,
        "r_ecl":0,"t_ecl":0,"f_ecl":0,
        "r_obs":0,"t_obs":0,"f_obs":0,
        #Normal to spangle
        "ns_equ":[0,0,1],
        "ns_ecl":[0,0,1],
        "ns_obs":[0,0,1],
        #Optical parameters
        "asp":0.0,
        "albedo_gray_normal":1,
        "tau_gray_optical":0,
        #Spangle state
        "unset":1, #State has not been set
        "visible":0, #The spangle is visible from observer
        "shadow":0, #The spangle is in the shadow of other spangle
        "illuminated":0, #The spangle is illuminated
        "transit":0, #The spangle is transiting
        "indirect":0, #The spangle is indirectly illuminated
        "occult":0, #The spangle is occulted by a light source
    }
)

In [5]:
class Spangling(object):
    
    def __init__(self,nspangles=0,body_hash=None):

        #Attributes
        self.nspangles=nspangles
        
        #Update default values
        self.defaults=deepcopy(SPANGLING_COLUMNS)
        
        if body_hash:
            self.defaults.update(dict(body_hash=body_hash))
        
        #Declare dataframe
        if self.nspangles>0:
            self.df=pd.DataFrame([list(self.defaults.values())]*nspangles,columns=self.defaults.keys())
        else:
            self.df=pd.DataFrame(columns=self.defaults.keys())

In [6]:
from IPython.display import display, HTML

if IN_JUPYTER:
    def test_fun(self):
        sg=Spangling(nspangles=10,body_hash="123")
        sg.df[["x_equ","y_equ","z_equ"]]=2*np.ones((10,3))
        Misc.print_df(sg.df.head(5))
        
        """
        self.assertEqual(self.P.Nr,8,True)
        self.assertEqual(np.isclose([P.physics.wrot],
                                    [2*np.pi/PlanetDefaults.physics["prot"]],
                                    rtol=1e-7),
                         [True]*1)
        self.assertRaises(AssertionError,lambda:Observer(primary="Nada"))
        """
        
    class Test(unittest.TestCase):pass
    Test.test_fun=test_fun
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

,body_hash,type,x_equ,y_equ,z_equ,x_ecl,y_ecl,z_ecl,x_obs,y_obs,z_obs,r_equ,t_equ,f_equ,r_ecl,t_ecl,f_ecl,r_obs,t_obs,f_obs,ns_equ,ns_ecl,ns_obs,asp,albedo_gray_normal,tau_gray_optical,unset,visible,shadow,illuminated,transit,indirect,occult
0,123,0,2.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1]","[0, 0, 1]","[0, 0, 1]",0.0,1,0,1,0,0,0,0,0,0
1,123,0,2.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1]","[0, 0, 1]","[0, 0, 1]",0.0,1,0,1,0,0,0,0,0,0
2,123,0,2.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1]","[0, 0, 1]","[0, 0, 1]",0.0,1,0,1,0,0,0,0,0,0
3,123,0,2.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1]","[0, 0, 1]","[0, 0, 1]",0.0,1,0,1,0,0,0,0,0,0
4,123,0,2.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1]","[0, 0, 1]","[0, 0, 1]",0.0,1,0,1,0,0,0,0,0,0


.
----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


### The end

In [7]:
Spangling.__doc__=Spangling_doc

--End--

### Documentation

- https://stackoverflow.com/questions/63977848/assign-multiple-columns-of-numpy-to-one-pandas-dataframe-column